# 배치란?
  * 데이터의 묶음
  * 묶음 대로 결과물이 계산 된다.
    * 100개 데이터를 한꺼번에 묶어서(배치를 만들어서) 입력을 했으면, 거기에 대한 결과물도 100개가 한꺼번에 나온다.
  * 배치를 적용한 Loss의 수식은?
  * `N`이면 `N`건에 대한 `CEE` 값을 구한 다음( 각각이라곤 했지만 한꺼번에 구해진다 )
  * 그 값들을 모두 더하고 `N`에 대한 평균을 구한다.

**배치를 적용한 CEE**
$$
CEE = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}\log{y_{nk}}
$$

# 미니배치란?
  * `MNIST`의 데이터의 개수는 60,000건
  * 신경망이 `MNIST`를 학습 하고, 거기에 대한 평가를 내릴 때 60,000건 모두에 대한 손실 함수의 합을 구해야 할까?
  * 데이터의 양이 굉장히 많은 경우에는 모든 데이터를 다 쓰는 것이 아니고, **데이터의 일부를 랜덤하게 추려서** 근사치로 이용할 수 있다.
  * 이 일부가 되는 데이터를 **미니배치**라고 한다.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# mnist 데이터세트 로딩
from tensorflow.keras import datasets
mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
TRAIN_IMAGE_SIZE = X_train.shape[0]

X_train = X_train.reshape(TRAIN_IMAGE_SIZE, -1)
X_train.shape, y_train.shape

((60000, 784), (60000,))

## y_train OHE

In [ ]:
y_train[:3]

array([5, 0, 4], dtype=uint8)

`y_train`이 One Hot Encoding이 되어있지 않은 상태
* ```python
  [5, 0, 4]
```
* ```python
  [[5],
   [0],
   [4]]
```
* ```python
 [[0,0,0,0,0,1,0,0,0,0],
   [1,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,1,0,0,0,0,0]]
```

In [ ]:
y_train_step1 = y_train.reshape(-1, 1)
y_train_step1.shape

(60000, 1)

In [ ]:
y_train_step1[:3]

array([[5],
       [0],
       [4]], dtype=uint8)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y_train_one_hot = OneHotEncoder().fit_transform(y_train_step1)
y_train_one_hot

<60000x10 sparse matrix of type '<class 'numpy.float64'>'
	with 60000 stored elements in Compressed Sparse Row format>

In [ ]:
y_train_one_hot_arr = y_train_one_hot.toarray()
y_train_one_hot_arr.shape

(60000, 10)

In [ ]:
y_train_one_hot_arr[:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

# 미니배치 구현하기

In [ ]:
import numpy as np

# 훈련 데이터 전체에서 무작위로 10장만 빼오기
train_size = X_train.shape[0] # 전체 훈련 데이터 세트 크기
batch_size = 10 # 미니 배치의 사이즈

# train_size에서 batch_size만큼 정수를 무작위로 선택
batch_mask = np.random.choice(train_size, batch_size) 

batch_mask

array([39898,  1301,  2545,  6608, 39529, 15622, 33191,  2762, 40509,
       29272])

In [ ]:
X_batch = X_train[batch_mask]
t_batch = y_train_one_hot_arr[batch_mask]

X_batch.shape, t_batch.shape

((10, 784), (10, 10))

$$
CEE = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}\log{y_{nk}}
$$

In [ ]:
# ver 1 - OHE이 되어 있는 형태
def cross_entropy_error_v1(y, t):

  delta = 1e-6
  # 배치를 사용하지 않은 경우
  if y.ndim == 1:
    # batch_size를 강제로 1로 지정한다. 
    # reshape으로..
    t = t.reshape(1, -1)
    y = y.reshape(1, -1)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y+delta)) / batch_size

In [ ]:
# ver 2 : One Hot Encoding이 되어있지 않으면?
def cross_entropy_error_v2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(q), t])) / batch_size

```python
np.log(y[np.arange(batch_size), t])

제일 안쪽에 있는 변수
batch_size : 5
t : [2, 7, 0, 9, 8]

batch_size를 이용하는 np.arange

np.arange(batch_size) : [0, 1, 2, 3, 4]

y[np.arange(batch_size), t]

y[[0,1,2,3,4],[2,7,0,9,8]]는

y[0, 2]
y[1, 7]
y[2, 0]
y[3, 9]
y[4, 8]

로 표현된다.

y는 5개의 배치 데이터에 대한 소프트맥스의 결과물

y = [[0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0],
     [0.0, 0.1, 0.7, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0]]

np.log(y[np.arange(batch_size), t])

0번째의 경우
np.log(y[0, 2])
np.log(0.7)
```